# Experiments on Real Data

Load libraries

In [ ]:
# External libs
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors
import pandas as pd
import numpy as np
import itertools
import os

import sys
if '../' not in sys.path:
    sys.path.append('../')

# lib for: ADM4, NPHC, WH
from tick.hawkes.inference import HawkesConditionalLaw, HawkesADM4, HawkesCumulantMatching
from tick.dataset import fetch_hawkes_bund_data
# lib for: Desync-MLE 
from desync_mhp.lib.inference import HawkesExpKernConditionalMLE

# Internal lib
import lib

Define general helper functions

In [ ]:
def plotmat_sidebyside(mats, grid, figsize=(5.5, 1.95), ticks=None, ytitle=None):
    """Plot matrices side-by-side with the same color scheme"""
    labels = list(mats.keys())
    mats = list(mats.values())
    n = len(mats)
    # Build colormap
    vmin = min(map(lambda A: A.min(), mats))
    vmax = max(map(lambda A: A.max(), mats))
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = 'plasma'
    # Plot matrices
    fig, axs = plt.subplots(*grid, figsize=figsize)
    axs = np.ravel(axs)
    for ax, M, label in zip(axs, mats, labels):
        plt.sca(ax)
        ax.invert_yaxis()
        ax.set_aspect(1.0)
        dim = len(M)
        X = np.tile(np.arange(dim+1)+0.5, (dim+1,1))
        Y = X.T
        p = plt.pcolormesh(X, Y, M, norm=norm, cmap=cmap)
        if ticks:
            plt.xticks(ticks)
            plt.yticks(ticks)
        p.cmap.set_over('white')
        p.cmap.set_under('black')
        plt.title(label, pad=10, y=ytitle)
        # create an axes on the right side of ax. The width of cax will be 5%
        # of ax and the padding between cax and ax will be fixed at 0.05 inch.
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        plt.colorbar(p, cax=cax)

    return fig

## 1. Load and Explore the dataset

Load the dataset

In [ ]:
events = fetch_hawkes_bund_data()

Explore inter-event time

In [ ]:
inter_event_time_list = list()
for raw_events_r in events:
    for raw_events_r_i in raw_events_r:
        t_diff = np.diff(np.unique(raw_events_r_i))
        assert t_diff.min() > 0
        inter_event_time_list.append(t_diff)
        
plt.hist(np.log10(np.hstack(inter_event_time_list)), bins=20);
plt.xlabel('(log) inter-event time')
plt.ylabel('Count');

Split the dataset. Use the first 5 days for hyperparameter tuning, use the remaining 15 days for evaluation.

In [ ]:
dev_events = events[:5]
train_events = events[5:]

---

## 2. Run Experiments Under Random Translation

### 2.1. Define helper functions for the experiments

#### 2.1.1. Define a helper function to add random translations to a given event dataset

In [ ]:
def generate_real_data_noisy(noise_scale, events, seed=None):
    noise_sampler = np.random.RandomState(seed)
    dim = len(events)
    # Add noise to the data
    noisy_events = list()
    for r, events_r in enumerate(events):
        noisy_events.append([])
        for i, events_r_i in enumerate(events_r):
            noisy_events_r_i = np.sort(events_r_i + noise_sampler.normal(scale=noise_scale, size=events_r_i.shape))
            noisy_events[r].append(noisy_events_r_i)
    return noisy_events

### 2.2. Define helper functions to run each learning method and perform hyperparameter tuning

#### 2.2.1. ADM4

Find best exponential decay providing the largest log-likelihood (i.e. the smallest loss)

In [ ]:
decay_search_range = np.logspace(3, 3.5, 10)

adm4_best_loss = np.inf
adm4_best_decay = np.nan
for test_decay in decay_search_range:
    adm4_dev = HawkesADM4(decay=test_decay, verbose=False)
    adm4_dev.fit(dev_events)
    loss = adm4_dev.objective(adm4_dev.coeffs)
    print(f"{test_decay:.2e} -> {loss:.2e}")
    if loss < adm4_best_loss:
        adm4_best_loss = loss
        adm4_best_decay = test_decay

Find best regularization weight

In [ ]:
C_search_range = np.logspace(0, 5, 6)

adm4_best_loss = np.inf
adm4_best_C = np.nan
for test_C in C_search_range:
    adm4_dev = HawkesADM4(decay=adm4_best_decay, C=test_C, verbose=False)
    adm4_dev.fit(dev_events)
    loss = adm4_dev.objective(adm4_dev.coeffs)
    print(f"{test_C:.2e} -> {loss:.2e}")
    if loss < adm4_best_loss:
        adm4_best_loss = loss
        adm4_best_C = test_C

Define helper function

In [ ]:
def run_adm4(events, return_extra=False):
    adm4 = HawkesADM4(decay=1291.0, C=1e3, verbose=False)
    adm4.fit(events)
    if return_extra:
        return adm4.adjacency.copy(), {'obj': adm4}
    return adm4.adjacency.copy()

#### 2.2.2. WH

Define helper function

In [ ]:
def run_wh(events):
    wh = HawkesConditionalLaw(
        claw_method="log", delta_lag=0.1, min_lag=5e-4, max_lag=500,
        quad_method="log", n_quad=10, min_support=1e-4, max_support=1, n_threads=4)
    wh.fit(events)
    return wh.get_kernel_norms()

#### 2.2.3. NPHC

Define helper function

In [ ]:
def run_nphc(events, H=4.19e-04):
    nphc = HawkesCumulantMatching(integration_support=H, 
                                  step=1e-2, solver='adam', max_iter=10000, 
                                  C=1e5, elastic_net_ratio=0.5, penalty='elasticnet', verbose=False)
    nphc.fit(events)
    return nphc.adjacency.copy()

#### 2.2.4. Desync-MLE

Define helper function

In [ ]:
def run_desyncmle(events, return_extra=False, verbose=False):    
    dim = len(events[0])
    # Desync-MLE (Use the same exponential decay and regularization scheme as ADM4)
    desyncmle = HawkesExpKernConditionalMLE(
        decay=1291.0,
        noise_penalty='l2', noise_C=1e5,
        hawkes_penalty='l1', hawkes_base_C=1e3, hawkes_adj_C=1e5, 
        solver='sgd', tol=1e-4, max_iter=1000,
        verbose=verbose
    )
    desyncmle.fit(events, end_time=max([max(map(max, ev)) for ev in events]),
                z_start=np.zeros(dim),
                theta_start=np.hstack((
                    0.01*np.ones(dim),
                    np.random.uniform(0.1, 0.2, size=dim**2),
                )),
                callback=None)
    desyncmle_adj = np.reshape(desyncmle.coeffs[2*dim:], (dim, dim))
    if return_extra:
        return desyncmle_adj, {'z': desyncmle.coeffs[:dim], 'mu': desyncmle.coeffs[dim:2*dim], 'obj': desyncmle}
    return desyncmle_adj

---

### 2.3. Run experiments for varying noise levels

##### First run each method on the original dataset

In [ ]:
adm4_base_adj = run_adm4(train_events)
wh_base_adj = run_wh(train_events)
nphc_base_adj = run_nphc(train_events)
desyncmle_base_adj = run_desyncmle(train_events)

Visualize the excitation matrix learned by each method

In [ ]:
plotmat_sidebyside(
    {
        '(a) ADM4': adm4_base_adj, 
        '(b) NPHC': nphc_base_adj, 
        '(c) WH': wh_base_adj,
        '(d) Desync-MLE': desyncmle_base_adj
    }, grid=(1, 4), figsize=(6.75, 3.25), ytitle=1.0, ticks=[1, 2, 3, 4])
plt.tight_layout()

##### Then run them on each method in randomly translated datasets for varying noise levels

In [ ]:
# Noise scale range to iterate over
noise_scale_range = np.logspace(np.log10(1e-1 /adm4_best_decay), np.log10(1e4 / adm4_best_decay), 10)
# Seed of random noise to iterate over
noise_seed_list = np.random.RandomState(703994370).randint(0, 2**32 - 1, size=20)
data = list()
# For each noise scale
for it, noise_scale in enumerate(noise_scale_range):
    # Simulate several datasets
    for it2, noise_seed in enumerate(noise_seed_list):
        print(f"Iter {it+1}/{len(noise_scale_range)} | {it2+1}/{len(noise_seed_list)} | noise_scale={noise_scale:.2e}")
        # Simulate noisy data
        noisy_train_events = generate_real_data_noisy(
            noise_scale=noise_scale, events=train_events, seed=noise_seed)
        # ADM4
        adm4_adj = run_adm4(noisy_train_events)
        # WH
        wh_adj = run_wh(noisy_train_events)
        # NPHC
        nphc_adj = run_nphc(noisy_train_events, H=1/1291.0+noise_scale)
        # Store estimations
        data.append({
            'noise_scale': noise_scale,
            'noise_seed': noise_seed,
            'adm4_adj': adm4_adj,
            'wh_adj': wh_adj,
            'nphc_adj': nphc_adj,
        })
df = pd.DataFrame(data)

---

## 3. Visualize the results

Compute the evaluation metrics

In [ ]:
method_queries = [
    (
        'adm4', 
        'ADM4'
    ),
    (
        'nphc', 
        'NPHC'
    ),
    (
        'wh',   
        'WH'
    )
]

metric_queries = [
    (
        'norm',   
        'Norm',           
        lambda y_test, y_true: np.linalg.norm(y_test.ravel(), ord=2)**2                  
    ),
    (
        'relerr', 
        'Relative Error', 
        lambda y_test, y_true: lib.utils.metrics.relerr(y_test, y_true, null_norm='min')
    )
]

base_adj_dict = {
    'adm4': adm4_base_adj,
    'wh': wh_base_adj,
    'nphc': nphc_base_adj,
}

for method, _ in method_queries:
    for suffix, _, func in metric_queries:
        df[f'{method}_{suffix}'] = df[f'{method}_adj'].apply(lambda adj: func(y_test=adj, y_true=base_adj_dict[method]))

df_plot = df.groupby('noise_scale').agg(['mean', 'std', 'count'])

Generate the plots

In [ ]:
for suffix, metric_label, _ in metric_queries:    
    print(metric_label, flush=True)
    plt.figure(figsize=(3.25, 3.25*0.5*0.85))
    plt.grid()
    for method, method_label in method_queries:
        yseries = df_plot[f'{method}_{suffix}']
        y = yseries['mean']
        yerr = yseries['std'] / np.sqrt(yseries['count'])
        plt.errorbar(df_plot.index, 
                     y=y, 
                     yerr=yerr, 
                     label=method_label, )
    
    if suffix == 'norm':
        plt.yscale('log')
    if suffix.startswith('relerr'):
        plt.yscale('log')
    
    plt.xscale('log')
    plt.xlabel(f'Noise scale $\sigma^2$')
    plt.ylabel(metric_label)
    plt.legend();